In [1]:
import numpy as np
import openpyxl
import copy
import matplotlib.pyplot as plt
import time

In [7]:
class Sudoku():
    
    def read_sudoku(self,name='Sudoku'):
        wb_obj = openpyxl.load_workbook(name+".xlsm")                   
        sheet_obj = wb_obj.active

        for i in range(1,10):
            for j in range(1,10):
                cell_obj = sheet_obj.cell(row = i, column = j)
                a = cell_obj.value
                if isinstance(a, int):
                    self.grid[i-1,j-1] = cell_obj.value
                    if a != 0:
                        self.grid_config[i-1,j-1] = 1
                        
    def print_sudoku(self):
        for i in range(len(self.grid)):
            line = ""
            if i == 3 or i == 6:
                print("---------------------")
            for j in range(len(self.grid[i])):
                if j == 3 or j == 6:
                    line += "| "
                if self.grid_config [i,j] == 1:
                    line += '\033[1m' + str(self.grid[i][j])+ '\033[0m' + " " 
                else:
                    line += str(self.grid[i][j])+ " "
            print(line)
        print("\n\n")
    
    def __init__(self):
        self.grid = np.zeros((9,9)).astype(int)
        self.grid_config = np.zeros((9,9)).astype(int)
        self.violations = np.zeros((9,9)).astype(int)
        self.square_violations = np.zeros(9).astype(int)
        self.read_sudoku()
        self.print_sudoku()
        self.solved = 0
        
    def feel_sudoku(self):
        for i in range(9):
            p = np.random.permutation([1,2,3,4,5,6,7,8,9])
            for j in range(9):
                p= np.delete(p, np.where(p == self.grid[i//3*3+j//3,j%3 + (3*i)%9] ))
            k = 0
            for j in range(9):
                if self.grid_config[i//3*3+j//3,j%3 + (3*i)%9] == 0:
                    self.grid[i//3*3+j//3,j%3 + (3*i)%9] = p[k]
                    if len(p)==1:
                        self.grid_config[i//3*3+j//3,j%3 + (3*i)%9] =1
                    k +=1
    def likelihood(self,sudoku):
        energy = 0
        for i in range(9):
            for j in range(9):
                value = sudoku[i,j]
                energy += np.sum(sudoku[i,:]==value) + np.sum(sudoku[:,j]==value) - 2            
        return energy
     
    def MCMC(self,T=0.15,max_iter=50000):
        self.feel_sudoku()
        number_of_acceptance = 0
        
        a = 0.95
        add = 1
        
        energy = self.likelihood(self.grid)
        if energy == 0:
            return _
        sudoku_trial = copy.deepcopy(self.grid)
        best_energy = energy
        evolution_of_energy = [energy]

        nb_of_false = 0

        for i in range(max_iter):

            square = np.random.randint(0,9)
            first_element = np.random.randint(0,9)
            while (self.grid_config[square//3*3+first_element//3,first_element%3 + (3*square)%9] ==1):
                square = np.random.randint(0,9)
                first_element = np.random.randint(0,9)
            second_element = np.random.randint(0,9)
            while (first_element == second_element or self.grid_config[square//3*3+second_element//3,second_element%3 + (3*square)%9]==1):
                second_element = np.random.randint(0,9)

            sudoku_trial = copy.deepcopy(self.grid)

            tmp = sudoku_trial[square//3*3+first_element//3,first_element%3 + (3*square)%9]
            sudoku_trial[square//3*3+first_element//3,first_element%3 + (3*square)%9] = sudoku_trial[square//3*3+second_element//3,second_element%3 + (3*square)%9]
            sudoku_trial[square//3*3+second_element//3,second_element%3 + (3*square)%9] = tmp



            e_trial = self.likelihood(sudoku_trial)
            if (i%100==0):
                evolution_of_energy.append(e_trial)

            U = np.random.uniform(0,1)

            if e_trial < energy or U < np.exp(-(e_trial-energy)/T) : 
                self.grid = copy.deepcopy(sudoku_trial)
                energy = e_trial
                number_of_acceptance +=1
                T = T * a
                nb_of_false = 0

            else:
                nb_of_false +=1
                if nb_of_false >= 100:
                    add = 10
                    T += add
                    nb_of_false = 0
         
            if energy < best_energy:
                best_energy = energy

            if best_energy == 0:
                self.solved = 1
                break
        return _
    
    def is_on_row(self,sudoku,k,i):
        for j in range(9):
            if (sudoku[i,j] == k):
                return True
        return False

    def is_on_column(self,sudoku,k,j):
        for i in range(9):
            if sudoku[i,j] == k:
                return True
        return False

    def is_on_block(self,sudoku,k,i,j):
        first_row = i - i%3
        first_col = j - j%3
        for row in range(first_row,first_row+3):
            for col in range(first_col,first_col+3):
                if sudoku[row,col] == k:
                    return True
        return False

    def is_valid(self,sudoku,k,i,j):
        if ( (self.is_on_row(sudoku,k,i)==False) and (self.is_on_column(sudoku,k,j)==False) and (self.is_on_block(sudoku,k,i,j) == False)):
            return True
        else:
            return False
        
    def solve_backtracking(self,sudoku,position=0):
        sudoku_trial = copy.deepcopy(sudoku)
        if position == 81:
            self.grid = copy.deepcopy(sudoku_trial)
            self.solved = 1
            return True
        i = position//9
        j = position%9
        if self.grid_config[i,j] == 1:
            self.solve_backtracking(sudoku_trial,position+1)
        else:
            for k in range(1,10):
                if self.is_valid(sudoku_trial,k,i,j):
                    sudoku_trial[i,j] = k
                    if self.solve_backtracking(sudoku_trial,position+1) == True:
                        return True
        return False
    
    def solve(self,method='backtracking'):
        start_time = time.time()
        if (method == 'backtracking'):
            self.solve_backtracking(self.grid)
        elif method == 'MCMC':
            self.MCMC()
        else:
            print("Unknown method")
        elapsed_time = time.time() - start_time
        if self.solved == 1:
            print('Solved in %0.2f seconds with %s method' %(elapsed_time,method))
            self.print_sudoku()
        else:
            print("No solution find in %0.2f seconds with %s method" %(elapsed_time,method))
            


In [8]:
sd = Sudoku()
sd.solve('backtracking')

1 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
---------------------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
---------------------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 



Solved in 0.02 seconds with backtracking method
1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 7 8 9 | 1 2 3 
7 8 9 | 1 2 3 | 4 5 6 
---------------------
2 1 4 | 3 6 5 | 8 9 7 
3 6 5 | 8 9 7 | 2 1 4 
8 9 7 | 2 1 4 | 3 6 5 
---------------------
5 3 1 | 6 4 2 | 9 7 8 
6 4 2 | 9 7 8 | 5 3 1 
9 7 8 | 5 3 1 | 6 4 2 





In [9]:
sd = Sudoku()
sd.solve('MCMC')

1 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
---------------------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
---------------------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 



Solved in 13.34 seconds with MCMC method
1 9 3 | 7 4 5 | 6 8 2 
2 6 5 | 8 3 9 | 4 7 1 
8 4 7 | 2 6 1 | 3 9 5 
---------------------
7 5 6 | 4 1 8 | 2 3 9 
9 8 1 | 3 5 2 | 7 4 6 
3 2 4 | 6 9 7 | 1 5 8 
---------------------
6 3 8 | 5 2 4 | 9 1 7 
4 7 9 | 1 8 6 | 5 2 3 
5 1 2 | 9 7 3 | 8 6 4 



